In [1]:
import praw
import pandas as pd
import time

# Initializing the Reddit API client with my unique credentials
reddit = praw.Reddit(client_id='9iNmvNiFvQs_V_cu24vphQ',
                     client_secret='wxmYcvLVW7E1dxZJ1HN1VLFkXlTb0A', 
                     password='Reddit!5',
                     user_agent='Jacquelyn', 
                     username='capstoneundercover')

# Specifing the desired subreddit and retrieving the posts
ci_subreddit = reddit.subreddit('ChronicIllness')
post_list = []
for post in ci_subreddit.new(limit=500):  # Retriveing the 500 newest posts at this given time
    post_list.append([post.title, post.selftext, post.score, post.num_comments, post.url])
    if len(post_list) >= 500:  # Stops after retrieving 500 posts to ensure that my data stays accurate and uniform throughout all locations
        break
    time.sleep(2)  # Using a 2-second delay between requests to ensure I don't go over Reddit's API rates
#this was taken on: march 13th, 2023
# Extracting post titles, bodies, scores (upvotes), number of comments, and url
post_titles = []
post_bodies = []
post_scores = []
post_comments = []
post_url = []
for post in post_list:
    post_titles.append(post[0])
    post_bodies.append(post[1])
    post_scores.append(post[2])
    post_comments.append(post[3])
    post_url.append(post[4])

# Creating a DataFrame
df = pd.DataFrame({'post_title': post_titles, 'post_body': post_bodies, 'post_score': post_scores,
                  'post_comments': post_comments, 'post_url': post_url})

# Saving the DataFrame to a CSV file
df.to_csv('ci_posts.csv', index=False)

In [2]:
# Ensuring that the previous code has successfully transferred to my CSV file
# This is not required, but I like the reassurance 
try:
    df.to_csv('ci_posts.csv', index=False)
    print("CSV file successfully saved.")
except Exception as e:
    print("Error occurred while saving CSV:", e)

CSV file successfully saved.


In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if isinstance(text, str):  # Checking to see if the input is a string
        tokens = word_tokenize(text)  
        tokens = [word.lower() for word in tokens if word.isalpha()]  # Removing punctuation and converting to lowercase
        custom_stop_words = ['got', 'sees', 'today', 'told', 'much', 'we', 'think',
                            'called', 'since']  # Add your custom stop words here
        stop_words.update(custom_stop_words)  # Update the stop_words set with custom stop words
        tokens = [word for word in tokens if word not in stop_words]  # Removing stop words
        return " ".join(tokens)
    else:
        return ""  # Return an empty string for NaN values

# Assuming 'data' is a DataFrame that has been defined earlier
data = df.dropna(subset=['post_body'])  # Removing rows with NaN values in 'post_body' column
data['clean_text'] = data['post_body'].apply(preprocess_text)

data.to_csv('cleaned_ci_data.csv', index=False)

In [13]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('cleaned_ci_data.csv') # Now you can use the 'df' DataFrame in your code

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Create an instance of the sentiment analyzer
sentimentAnalyzer = SentimentIntensityAnalyzer()
for post in df.index:
    if isinstance(df['clean_text'][post], str):
        print(df['clean_text'][post][0:100])
        print(sentimentAnalyzer.polarity_scores(df['clean_text'][post]))

starting first job monday nervous going balance many doctor appointments company gave offer hours ap
{'neg': 0.113, 'neu': 0.835, 'pos': 0.052, 'compound': -0.3612}
anyone get jealous healthy people sick physical pain people actually care instance father went hospi
{'neg': 0.291, 'neu': 0.496, 'pos': 0.214, 'compound': -0.8496}
hi good ten years started debilitating chronic pain generally unwell finally appointment rheumatolog
{'neg': 0.153, 'neu': 0.591, 'pos': 0.255, 'compound': 0.6705}
fighting answers chronic illness years recently amazing specialist truly struggling gaslit last one 
{'neg': 0.197, 'neu': 0.557, 'pos': 0.246, 'compound': 0.8752}
man want complain many chronic things sinus infections around fall mold painful procedures maybe sur
{'neg': 0.331, 'neu': 0.602, 'pos': 0.067, 'compound': -0.9423}
cried phone schedule physical therapy referral expired days referrals blood draw orders hospital gro
{'neg': 0.237, 'neu': 0.611, 'pos': 0.152, 'compound': -0.7783}
ssri zoloft 

In [15]:
from textblob import TextBlob

# Replace NaN values in 'clean_text' column with an empty string
df['clean_text'] = df['clean_text'].fillna('')

# Add a new column to the DataFrame to store the sentiment scores
df['sentiment_scores'] = df['clean_text'].apply(lambda text: TextBlob(str(text)).sentiment.polarity)

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_ci_data_with_sentiment.csv', index=False)

In [18]:
import pandas as pd

# Loading the CSV file into the DataFrame again
df = pd.read_csv('updated_ci_data_with_sentiment.csv')

# Exporting the DataFrame to an Excel file located on my computer for easier viewing
df.to_excel('final_ci_posts.xlsx', index=False)